## Agent in Team 
- A single Agent can only say create a short story for us.
- but with a team whre many agents work together towards a common goal they can help us in writing or even helping to review, edit etc.

In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

from dotenv import load_dotenv 
import os 
load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")

In [2]:
model_client = OpenAIChatCompletionClient(
    model="gemini-1.5-flash-8b",
    api_key=api_key)

In [ ]:
from autogen_agentchat.agents import AssistantAgent

dsa_solver = AssistantAgent(
    name = 'Complex_DSA_Solver',
    model_client=model_client,
    description='A DSA solver',
    system_message="You give code in python to solve complex DSA problems. Give under 100 words"
)

code_reviewer = AssistantAgent(
    name = 'CODE_REVEIWER',
    model_client=model_client,
    description='A Code Reviewer',
    system_message="You review the code given by the complex_dsa_solver and make sure it is optimized.Give under 10 words"
)

code_editor = AssistantAgent(
    name = 'CODE_EDITOR',
    model_client=model_client,
    description='A Code editor',
    system_message="You make the code easy to understand and add comments wherever required.Give under 10 words"
)


In [4]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage

team = RoundRobinGroupChat(
    participants=[dsa_solver,code_reviewer,code_editor],
    max_turns=4 # -----># maximum number of Message before it stops between the agents.
)

async def test_team():
    task = TextMessage(content='Write a simple Hello world code ?',source='User')
    
    result = await team.run(task=task)

    for each_agent_message in result.messages:
        print(f'{each_agent_message.source} : {each_agent_message.content}' )
        print('\n \n')


await test_team()

User : Write a simple Hello world code ?

 

Complex_DSA_Solver : ```python
print("Hello, world!")
```


 

CODE_REVEIWER : That's a perfectly good, optimized "Hello, world!" program.


 

CODE_EDITOR : OK.  Is there anything else?


 

Complex_DSA_Solver : Yes,  what else would you like to do?  Please provide a specific task or problem.


 



## Agent Streams

In [7]:
await team.reset()

from autogen_agentchat.base import TaskResult

async for message in team.run_stream(task="Write a simple Hello world code ?"):
    if isinstance(message, TaskResult):
        print("Stop reason : ",message.stop_reason)
    else:
        print(message)


source='user' models_usage=None metadata={} content='Write a simple Hello world code ?' type='TextMessage'
source='Complex_DSA_Solver' models_usage=RequestUsage(prompt_tokens=25, completion_tokens=12) metadata={} content='```python\nprint("Hello, world!")\n```\n' type='TextMessage'
source='CODE_REVEIWER' models_usage=RequestUsage(prompt_tokens=44, completion_tokens=16) metadata={} content='That\'s a perfectly good, optimized "Hello, world!" program.\n' type='TextMessage'
source='CODE_EDITOR' models_usage=RequestUsage(prompt_tokens=54, completion_tokens=9) metadata={} content='OK.  Is there anything else?\n' type='TextMessage'
source='Complex_DSA_Solver' models_usage=RequestUsage(prompt_tokens=62, completion_tokens=33) metadata={} content='Yes,  what kind of "else" are you looking for?  More code examples?  A different type of task?  Please clarify your request.\n' type='TextMessage'
Stop reason :  Maximum number of turns 4 reached.


In [8]:
from autogen_agentchat.ui import Console
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a short poem about the fall season."))  # Stream the messages to the console.


---------- TextMessage (user) ----------
Write a short poem about the fall season.
---------- TextMessage (Complex_DSA_Solver) ----------
The leaves, a fiery, crimson hue,
Descend, a whispered, autumn adieu.
Crisp air, a gentle, rustling sigh,
As summer fades and winter nigh.
Cool breezes dance, a golden grace,
Nature's beauty, time and space.

---------- TextMessage (CODE_REVEIWER) ----------
A lovely autumn poem!  Well-written.

---------- TextMessage (CODE_EDITOR) ----------
Thanks!  I appreciate it.

---------- TextMessage (Complex_DSA_Solver) ----------
You're welcome!  I'm glad you enjoyed it.



TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='Complex_DSA_Solver', models_usage=RequestUsage(prompt_tokens=27, completion_tokens=58), metadata={}, content="The leaves, a fiery, crimson hue,\nDescend, a whispered, autumn adieu.\nCrisp air, a gentle, rustling sigh,\nAs summer fades and winter nigh.\nCool breezes dance, a golden grace,\nNature's beauty, time and space.\n", type='TextMessage'), TextMessage(source='CODE_REVEIWER', models_usage=RequestUsage(prompt_tokens=92, completion_tokens=11), metadata={}, content='A lovely autumn poem!  Well-written.\n', type='TextMessage'), TextMessage(source='CODE_EDITOR', models_usage=RequestUsage(prompt_tokens=97, completion_tokens=8), metadata={}, content='Thanks!  I appreciate it.\n', type='TextMessage'), TextMessage(source='Complex_DSA_Solver', models_usage=RequestUsage(prompt_tokens=104, completion_tokens=15), metadata=

In [18]:
add_1_agent_first = AssistantAgent(
    name = 'Agent1',
    model_client=model_client,
    system_message="Add 1 to the 0 and Give result as output"
)

add_1_agent_second = AssistantAgent(
    name = 'Agent2',
    model_client=model_client,
    system_message="Add 1 to the number given by previos agent and Give result as output"
)

add_1_agent_third = AssistantAgent(
    name = 'Agent3',
    model_client=model_client,
    system_message="Add 1 to the number given by previos agent and Give result as output"
)


In [19]:
increament_team = RoundRobinGroupChat(
    participants=[add_1_agent_first,add_1_agent_second,add_1_agent_third],
    max_turns= 3
)

In [ ]:
await increament_team  .reset()  # Reset the team for a new task.
await Console(increament_team.run_stream())  # Stream the messages to the console.

Error processing publish message for Agent1_efa12a46-ac89-4f42-a681-1cbe4d9dc751/efa12a46-ac89-4f42-a681-1cbe4d9dc751
Traceback (most recent call last):
  File "d:\_AgenticAI\autogen\autogen-env\Lib\site-packages\autogen_core\_single_threaded_agent_runtime.py", line 604, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\_AgenticAI\autogen\autogen-env\Lib\site-packages\autogen_core\_base_agent.py", line 119, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\_AgenticAI\autogen\autogen-env\Lib\site-packages\autogen_agentchat\teams\_group_chat\_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\_AgenticAI\autogen\autogen-env\Lib\site-packages\autogen_core\_routed_agent.py", line 485, in on_message_impl
    return await h(self, message, ctx)
 

RuntimeError: BadRequestError: Error code: 400 - [{'error': {'code': 400, 'message': '* GenerateContentRequest.contents: contents is not specified\n', 'status': 'INVALID_ARGUMENT'}}]
Traceback:
Traceback (most recent call last):

  File "d:\_AgenticAI\autogen\autogen-env\Lib\site-packages\autogen_agentchat\teams\_group_chat\_chat_agent_container.py", line 79, in handle_request
    async for msg in self._agent.on_messages_stream(self._message_buffer, ctx.cancellation_token):

  File "d:\_AgenticAI\autogen\autogen-env\Lib\site-packages\autogen_agentchat\agents\_assistant_agent.py", line 827, in on_messages_stream
    async for inference_output in self._call_llm(

  File "d:\_AgenticAI\autogen\autogen-env\Lib\site-packages\autogen_agentchat\agents\_assistant_agent.py", line 955, in _call_llm
    model_result = await model_client.create(
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "d:\_AgenticAI\autogen\autogen-env\Lib\site-packages\autogen_ext\models\openai\_openai_client.py", l